In [1]:
import pandas as pd
import json
from tqdm import tqdm

# импортируем библиотеку для работы с картами
from keplergl import KeplerGl

/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [73]:
f = open('20220511T174759.log')
line = f.readline()

In [74]:
line

'11052022:160120;cherepovets;ak1456_2058;#7;bus;59.13450241088867;37.923988342285156;15.17;192;1;;е440хк_35 [БВ (Н)];7 [ул. олимпийская - > - ул. наседкина]\n'

[парсер .log](https://medium.datadriveninvestor.com/importing-log-files-with-ease-cb855982fa8c)

In [75]:
a = []

f = open('20220511T174759.log')
line = f.readline()
while line:
    line = f.readline()
    a.append(line)
    
f.close()    

In [80]:
df = pd.DataFrame([sub.split(";") for sub in a])

In [82]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,11052022:160119,cherepovets,ak1456_1881,#,bus,59.1339225769043,37.97321701049805,0.00,0,0,,е254хк_35 [БВ (Н)],\n
1,11052022:160117,cherepovets,ak1456_2424,#7,bus,59.12198257446289,37.97989273071289,9.44,8,1,,е401хк_35 [БВ (Н)],7 [ул. олимпийская - > - ул. наседкина]\n
2,11052022:160117,cherepovets,ak1456_1946,#27,bus,59.096702575683594,37.9358024597168,28.49,280,1,,е387хк_35 [БВ (Н)],27 [ул. монт - клер - > - доменная]\n
3,11052022:160118,cherepovets,ak1456_1892,#8,bus,59.126468658447266,37.91294479370117,17.76,103,1,,е329хк_35 [БВ (Н)],8 [доменная - > - ул. красная]\n
4,11052022:160115,cherepovets,ak1456_2408,#18,bus,59.1187744140625,37.90309143066406,0.00,13,1,,е407хк_35 [БВ (Н)],18 [ул. рыбинская (2) -> пр. победы 2]\n


Наименование колонок:

- `TIMESTAMP` - время в UTC в формате дата:время
- `CLID` - vologda или cherepovets
- `UUID` - уникальный идентификатор ТС в формате id_перевозчика + "_" + id_ТС_в базе_перевозчика
- `ROUTE` - "#" + номер маршрута 
- `VEHICLE_TYPE` - 'bus', 'tramway', 'minibus', 'trolleybus'
- `LATITUDE` – широта
- `LONGITUDE` – долгота
- `SPEED` - текущая скорость
- `DIRECTION` - азимут
- `PRODUCTION` - признак производственного рейса. Когда "0" - скорее всего автобус без пассажиров. В Яндекс передаются только данные с флагом "1"
- `GARANGE_NUMBER` - опционально "человеческий" идентификатор ТС для перевозчика
- `REG_NUMBER` - госномер ТС, есть не всегда
- `ROUTE_DESCR` - текстовый идентификатор траектории маршрута в одном направлении. Нет стандартного формата, можно использовать только как "хэш", как ключ для группировки точек по траекториям

In [88]:
new_name_columns = ['TIMESTAMP', 
                    'CLID', 
                    'UUID', 
                    'ROUTE', 
                    'VEHICLE_TYPE', 
                    'LATITUDE', 
                    'LONGITUDE', 
                    'SPEED', 
                    'DIRECTION', 
                    'PRODUCTION', 
                    'GARANGE_NUMBER', 
                    'REG_NUMBER', 
                    'ROUTE_DESCR'
                   ]

new_name_columns = list(map(lambda x: x.lower(), new_name_columns))

df.columns = new_name_columns

In [96]:
df.head()

,timestamp,clid,uuid,route,vehicle_type,latitude,longitude,speed,direction,production,garange_number,reg_number,route_descr
0,11052022:160119,cherepovets,ak1456_1881,#,bus,59.1339225769043,37.97321701049805,0.00,0,0,,е254хк_35 [БВ (Н)],\n
1,11052022:160117,cherepovets,ak1456_2424,#7,bus,59.12198257446289,37.97989273071289,9.44,8,1,,е401хк_35 [БВ (Н)],7 [ул. олимпийская - > - ул. наседкина]\n
2,11052022:160117,cherepovets,ak1456_1946,#27,bus,59.096702575683594,37.9358024597168,28.49,280,1,,е387хк_35 [БВ (Н)],27 [ул. монт - клер - > - доменная]\n
3,11052022:160118,cherepovets,ak1456_1892,#8,bus,59.126468658447266,37.91294479370117,17.76,103,1,,е329хк_35 [БВ (Н)],8 [доменная - > - ул. красная]\n
4,11052022:160115,cherepovets,ak1456_2408,#18,bus,59.1187744140625,37.90309143066406,0.00,13,1,,е407хк_35 [БВ (Н)],18 [ул. рыбинская (2) -> пр. победы 2]\n


# Создание карты

In [94]:
with open('config.json') as f:
    config = json.load(f)

_map = KeplerGl()
_map.config = config

_map.add_data(data=df_draw, name='yelyi5ow8')

_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['yelyi5ow8'], 'id': '3ynfdw2…